<a href="https://colab.research.google.com/github/timofeyegorov/gitTutorial/blob/main/%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B2%D0%B5%D0%B1_%D1%81%D0%BA%D1%80%D0%B0%D0%BF%D0%BF%D0%B8%D0%BD%D0%B3_%D0%B8_%D0%BF%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_%D1%81%D0%B0%D0%B9%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Немного теории

In [ ]:
from IPython.display import Image # Библиотека для отображения картинок

display(Image(url='https://prowebscraper.com/blog/wp-content/uploads/2017/11/Web_Scraping_for_Non-Programmers.png', 
              width = 1400) # Задаем ширину картинки
       ) 

**Веб-скраппинг** - получения веб-данных путем извлечения их со страниц веб-ресурсов. Некоторые сайты предоставляют специальный API для программного доступа к своим данным.


**Парсинг** - анализ и обработка полученных данных.

Минусы веб-скраппинга.
- Он хрупкий (веб-страницы, которые вы очищаете, могут часто меняться).
- Это может быть запрещено (некоторые веб-приложения имеют политику против скрапинга).
- Это может быть медленным и экспансивным (если вам нужно забирать и пропускать много шума).
__________________________________________________
Вебскраппинг делится на статический и динамический.

**Статический** - получает статический код страницы. Игнорирует JavaScript. Можно выполнить с помощью библиотек requests и BeautifulSoup. С помощью requests отправляют запрос на получение информации с веб-страницы, а BeautifulSoup нужен, чтобы распарсить полученную html-разметку.

**Динамический** - использует фактический браузер и позволяет JavaScript работать. Иногда вам необходимо автоматизировать браузер, моделируя пользователя, чтобы получить необходимый контент. Используется библиотека Selenium. Она имтирует работе браузера, то можно нажимать на кнопки, заполнять формы.

# Скраппинг базы автомобилей с Авто.ру с BeautifulSoup

**Для чего нужна задержка для скраппинга.**

Если вы будете парсить более-менее серьезные сайты - можете столкнутся с защитой от парсинга.
Например, если парсить какой-то сайт достаточно длительное время и слать запросы к нему очень часто - этот сайт может вас заблокировать по ip и выдать капчу. Кроме того, это несет нагрузку на сайт, и хорошо бы поставить задержку, чтобы сильно не нагружть сервер.
Основной принцип обхода защиты от парсинга - ваш парсер должен вести себя так, как вел бы себя человек, зашедший на сайт. Для каждого сайта задержка подбирается индивидуально.

In [ ]:
from bs4 import BeautifulSoup  # импорт класса BeautifulSoup из модуля bs4 Для парсинга страниц
import requests                # Для отправки запросов к сайту
from requests import get       # Запрос на получение данных
from tqdm import tqdm
import urllib.request          # библиотка для скачивания изображений
import time
import random
import os
import pandas as pd

In [ ]:
# Ссылка на страницу, которую парсим, куда динамически подставляем номер страницы
url = f'https://auto.ru/novosibirsk/cars/all/?page=1'
response = get(url)         # Получаем статус ответа сервера, 200 - все ок

response.encoding                                             # Смотрим какая кодировка используется
content = response.content                                    # Получаем байтовое представление данных для нетекстовых запросов
html_soup = BeautifulSoup(content.decode('utf-8','ignore'))   # Декодируем данные, игнорируя ошибки в символах

In [ ]:
html_soup

In [ ]:
cars_data = html_soup.find_all('div', class_ = 'ListingItem-module__description')

In [ ]:
cars_data[0]

<div class="ListingItem-module__description"><div class="ListingItem-module__column ListingItem-module__columnLeft"><div class="ListingItem-module__columnRowFlex"><div class="ListingItem-module__columnCellSummary"><h3 class="ListingItemTitle-module__container ListingItem-module__title"><a class="Link ListingItemTitle-module__link" href="https://auto.ru/cars/used/sale/dongfeng/s30/1103140890-71e6406c/" target="_blank">DongFeng S30<div><div class="ListingItemTitle-module__clicker"></div></div></a></h3><div class="ListingItemTechSummaryDesktop ListingItem-module__techSummary"><div class="ListingItemTechSummaryDesktop__column"><div class="ListingItemTechSummaryDesktop__cell">1.6 л / 117 л.с. / Бензин</div><div class="ListingItemTechSummaryDesktop__cell">механика</div><div class="ListingItemTechSummaryDesktop__cell">седан</div></div><div class="ListingItemTechSummaryDesktop__column"><div class="ListingItemTechSummaryDesktop__cell">передний</div><div class="ListingItemTechSummaryDesktop__cel

In [ ]:
start_time = time.time()        # Засекаем время выполнения кода в ячейке
cars = []                       # Список для хранения спарсенных данных для каждого объявления
count = 1                       # Счетчик страниц сайта, начинаем извлекать данные с первой страницы

while count <= 5:               # Устанавливаем максимальное количество страниц с которых будем собирать данные
    print(f'Страница {count}')  # Информация для вывода о том, какая страница сейчас парсится

    # Ссылка на страницу, которую парсим, куда динамически подставляем номер страницы
    url = f'https://auto.ru/novosibirsk/cars/all/?page={count}'
    response = get(url)         # Получаем статус ответа сервера, 200 - все ок
    
    response.encoding                                             # Смотрим какая кодировка используется
    content = response.content                                    # Получаем байтовое представление данных для нетекстовых запросов
    html_soup = BeautifulSoup(content.decode('utf-8','ignore'))   # Декодируем данные, игнорируя ошибки в символах

    #html_soup = BeautifulSoup(response.text, 'html.parser') - заменяет предыдущие 3 строки кода, если нет проблем с кодировкой


    cars_data = html_soup.find_all('div', class_ = 'ListingItem-module__description') # Находим все div блоки с классом 'ListingItem-module__main'
                                                                               # Метод возвращает список, где каждый div блок - это отдельное объявление

    if cars_data != []:                                                        # Проверяем, что данные есть
      cars.extend(cars_data)                                                   # Добавляем в список данных данные всех объявлений с текущей страницы
      print(f'Со страницы спарсено {len(cars_data)} объявлений, всего объявлений спарсено: {len(cars)}')

      # Генерируем задержку до следующего запроса от 1 до 11 секунд
      value = random.random()                                               # Генерируем случайное число от 0 до 1
      time_sleep = 1 + value*10                                             # Добавляем случайное число к единице
      print(f'Задержка до следующего запроса: {round(time_sleep, 1)} сек')
      time.sleep(time_sleep)                                                # Откладываем исполнение кода на time_sleep секунд
      print('--------------------------------')

    else:
      print('empty')                                                        # Если данные не справились - прекращаем цикл
      break  
    count += 1                                                              # Увеличиваем счетчик на 1 (переходим к парсингу следующей страницы)

end_time = time.time() - start_time                                         # Считаем сколько времени ушло на парсинг
print()
print(f'Время извлечения данных: {round(end_time/60, 1)} минут')

Страница 1
Со страницы спарсено 38 объявлений, всего объявлений спарсено: 38
Задержка до следующего запроса: 1.1 сек
--------------------------------
Страница 2
Со страницы спарсено 38 объявлений, всего объявлений спарсено: 76
Задержка до следующего запроса: 4.0 сек
--------------------------------
Страница 3
Со страницы спарсено 38 объявлений, всего объявлений спарсено: 114
Задержка до следующего запроса: 9.9 сек
--------------------------------
Страница 4
Со страницы спарсено 38 объявлений, всего объявлений спарсено: 152
Задержка до следующего запроса: 5.9 сек
--------------------------------
Страница 5
Со страницы спарсено 38 объявлений, всего объявлений спарсено: 190
Задержка до следующего запроса: 8.8 сек
--------------------------------

Время извлечения данных: 0.7 минут


In [ ]:
cars[:3]

[<div class="ListingItem-module__description"><div class="ListingItem-module__column ListingItem-module__columnLeft"><div class="ListingItem-module__columnRowFlex"><div class="ListingItem-module__columnCellSummary"><h3 class="ListingItemTitle-module__container ListingItem-module__title"><a class="Link ListingItemTitle-module__link" href="https://auto.ru/cars/used/sale/lexus/rx/1103095632-ac1bd181/" target="_blank">Lexus RX  III Рестайлинг 350<div><div class="ListingItemTitle-module__clicker"></div></div></a></h3><div class="ListingItemTechSummaryDesktop ListingItem-module__techSummary"><div class="ListingItemTechSummaryDesktop__column"><div class="ListingItemTechSummaryDesktop__cell">3.5 л / 277 л.с. / Бензин</div><div class="ListingItemTechSummaryDesktop__cell">автомат</div><div class="ListingItemTechSummaryDesktop__cell">внедорожник 5 дв.</div></div><div class="ListingItemTechSummaryDesktop__column"><div class="ListingItemTechSummaryDesktop__cell">полный</div><div class="ListingItemT

In [ ]:
count = 0       # Обнуляем счетчик, будем извлекать нужную информацию с нулевого элемента
data = []       # Список для хранения фильтрованной информации по каждому объявлению
skip_ads = 0    # Счетчик пропущенных объявления

# Задаем, какое кол-во объявлений спарсим
while count <= (len(cars)-1):
  try:                                                                            # Применяем try - except, чтобы обработать ошибку, когда не указана цена авто
    info = cars[int(count)]                                                       # Получаем блок div по объявлению
    title = info.find('a', {'class':'Link ListingItemTitle-module__link'}).text   # Находим тег 'a' с классом 'ListingItem-module__main', извлекаем текстовую информацию
    price = info.find('div', {'class':'ListingItemPrice-module__content'}).text   # Находим тег div с классом 'ListingItem-module__main', извлекаем текстовую информацию

    other_info = info.find_all('div', {'class':'ListingItemTechSummaryDesktop__cell'}) # Находим ВСЕ теги div с классом 'ListingItemTechSummaryDesktop__cell' (их 5 штук), метод возвращает список

    # Для каждого из пяти тегов div с классом 'ListingItemTechSummaryDesktop__cell' получаем текстовую информацию
    characteristics = str(other_info[0].get_text())
    transmission = str(other_info[1].get_text())
    body = str(other_info[2].get_text())
    drive = str(other_info[3].get_text())
    color = str(other_info[4].get_text())

    # Выводим данные которые спарсили для каждого объявления
    print(count, '. ', title, ' - ', price, ' - ', characteristics, ' - ', transmission, ' - ', body, ' - ', drive, ' - ', color, sep = '')

    temp_list = [title, price, characteristics, transmission, body, drive, color] # Во временный список добавляем все спарсенные данные по текущему объявлению
    data.append(temp_list)                                                        # Добавляем данные по текущему объявлению в дата для дальнейшего преобразования в пандас ДатаФрейм
  except AttributeError:
    skip_ads +=1                                                                # Считаем кол-во пропущенных объявлений
    print(count)
  count += 1                                                                    # Увеличиваем счетчик для перехода к парсингу следующего объявления
print()
print(f'Пропущено {skip_ads} объявлений')  

0. Volkswagen Polo VI - от 1 100 500 ₽ - 1.6 л / 110 л.с. / Бензин - механика - лифтбек - передний - Status
1. Mercedes-Benz G-Класс AMG  I (W463) Рестайлинг 3 63 AMG - 6 850 000 ₽ - 5.5 л / 571 л.с. / Бензин - автомат - внедорожник 5 дв. - полный - чёрный
2. Land Rover Range Rover Evoque I Рестайлинг - 2 200 000 ₽ - 2.0 л / 180 л.с. / Дизель - автомат - внедорожник 3 дв. - полный - серый
3. Toyota RAV4 V (XA50) - от 3 098 040 ₽ - 2.5 л / 199 л.с. / Бензин - автомат - внедорожник 5 дв. - полный - Комфорт
4. Lincoln Aviator II - 9 849 000 ₽ - 3.0 л / 494 л.с. / Гибрид - автомат - внедорожник 5 дв. - полный - серый
5. Toyota Camry VI (XV40) - 777 000 ₽ - 2.4 л / 167 л.с. / Бензин - автомат - седан - передний - чёрный
6. ГАЗ 21 «Волга»  21В - 2 200 000 ₽ - 2.5 л / 70 л.с. / Бензин - механика - седан - задний - чёрный
7
8. Lexus ES  VII 250 - 3 350 000 ₽ - 2.5 л / 200 л.с. / Бензин - автомат - седан - передний - серый
9. Audi Q7 II (4M) - 4 150 000 ₽ - 3.0 л / 249 л.с. / Дизель - автомат -

In [ ]:
columns = ['title', 'price', 'characteristics', 'transmission', 'body', 'drive', 'color']
df = pd.DataFrame(data, columns=columns)

In [ ]:
df

,title,price,characteristics,transmission,body,drive,color
0,Honda Stepwgn V Spada,1 920 000 ₽,1.5 л / 150 л.с. / Бензин,вариатор,минивэн,полный,фиолетовый
1,Land Rover Range Rover IV,6 100 000 ₽,4.4 л / 339 л.с. / Дизель,автомат,внедорожник 5 дв.,полный,серебристый
2,Mercedes-Benz CLS III (C257) 450,4 600 000 ₽,3.0 л / 367 л.с. / Бензин,автомат,седан,полный,серый
3,Lincoln Aviator II,9 849 000 ₽,3.0 л / 494 л.с. / Гибрид,автомат,внедорожник 5 дв.,полный,серый
4,Toyota Camry VI (XV40),777 000 ₽,2.4 л / 167 л.с. / Бензин,автомат,седан,передний,чёрный
...,...,...,...,...,...,...,...
180,Audi Q5 I (8R) Рестайлинг,1 420 000 ₽,2.0 л / 225 л.с. / Бензин,автомат,внедорожник 5 дв.,полный,белый
181,Chevrolet Cruze I Рестайлинг,620 000 ₽,1.6 л / 109 л.с. / Бензин,автомат,хэтчбек 5 дв.,передний,чёрный
182,Haval H9 I Рестайлинг,2 615 000 ₽,2.0 л / 245 л.с. / Бензин,автомат,внедорожник 5 дв.,полный,чёрный
183,Subaru Forester III Рестайлинг,869 000 ₽,2.0 л / 150 л.с. / Бензин,автомат,внедорожник 5 дв.,полный,синий


# Скраппинг изображений с BeautifulSoup

In [ ]:
link_root = f'http://www.motorpage.ru'                                          # Корневая ссылка на сайт, откуда будем парсить объявления
response = get(link_root + '/media/photos.html')                                # Получение информации о галерее изображений
html_soup = BeautifulSoup(response.text)                                        # Получаем разметку html-страницы для последующей обработки

cars_data = html_soup.find_all('a', {'class': 'col-xs-12 col-sm-6 zero-padding'})
cars_data

In [ ]:
cars_data[:3]

[<a class="col-xs-12 col-sm-6 zero-padding" href="/Acura/photos.html"> <figure> <img alt="Фото машин Acura" src="http://cdn.motorpage.ru/Photos/300/240A1.jpg"/> <figcaption class="caption-photos">365</figcaption> </figure> <div> <div> <img alt="Логотип Acura" src="http://cdn.motorpage.ru/AutoLogoMid/Acura_1.jpg"/> <h3>Acura</h3> <p class="hidden visible-md visible-lg">Моделей: 7</p> </div> </div> </a>,
 <a class="col-xs-12 col-sm-6 zero-padding" href="/AlfaRomeo/photos.html"> <figure> <img alt="Фото машин Alfa Romeo" src="http://cdn.motorpage.ru/Photos/300/Giulietta_11.jpg"/> <figcaption class="caption-photos">241</figcaption> </figure> <div> <div> <img alt="Логотип Alfa Romeo" src="http://cdn.motorpage.ru/AutoLogoMid/Alfa_Romeo_1.jpg"/> <h3>Alfa Romeo</h3> <p class="hidden visible-md visible-lg">Моделей: 15</p> </div> </div> </a>,
 <a class="col-xs-12 col-sm-6 zero-padding" href="/Alpina/photos.html"> <div> <div> <img alt="Логотип Alpina" src="http://cdn.motorpage.ru/AutoLogoMid/Alpin

In [ ]:
brand_links = [link['href'] for link in cars_data]   
brand_links[:3]

['/Acura/photos.html', '/AlfaRomeo/photos.html', '/Alpina/photos.html']

In [ ]:
brand_names = [link[1:link.rfind('/')].capitalize() for link in brand_links] 

In [ ]:
('acura').capitalize()

'Acura'

In [ ]:
brand_dict = {k:v for k, v in zip(brand_names, brand_links)}                    # Создаем словарь, где ключ - название бренда, значение - окончание ссылки на страницу с фото
number_cars = html_soup.find_all('figcaption', {'class': 'caption-photos'})     # По каждому авто парсим информацию по количеству фото
cars_values = [int(cars.text) for cars in number_cars]        

In [ ]:
brand_numbers = list(zip(brand_names, cars_values)) 
brand_numbers[:3]

[('Acura', 365), ('Alfaromeo', 241), ('Alpina', 38)]

In [ ]:
# Соберем информацию в текстовом виде о наличии фотографий для каждой марки авто

link_root = f'http://www.motorpage.ru'                                          # Корневая ссылка на сайт, откуда будем парсить объявления
response = get(link_root + '/media/photos.html')                                # Получение информации о галерее изображений
html_soup = BeautifulSoup(response.text)                                        # Получаем разметку html-страницы для последующей обработки

cars_data = html_soup.find_all('a', {'class': 'col-xs-12 col-sm-6 zero-padding'})  # Находим блоки с информацией о количестве фото по каждой марке авто

brand_links = [link['href'] for link in cars_data]                              # Получение окончания ссылки для каждого бренда, например, Acura/photos.html (полный адрес: http://www.motorpage.ru/Acura/photos.html)
brand_names = [link[1:link.rfind('/')].capitalize() for link in brand_links]    # Получение имени каждого бренда из окончания ссылки

brand_dict = {k:v for k, v in zip(brand_names, brand_links)}                    # Создаем словарь, где ключ - название бренда, значение - окончание ссылки на страницу с фото
number_cars = html_soup.find_all('figcaption', {'class': 'caption-photos'})     # По каждому авто парсим информацию по количеству фото
cars_values = [int(cars.text) for cars in number_cars]                          # Получаем тексотовую информацию о количестве фото по каждому авто из number_cars

brand_numbers = list(zip(brand_names, cars_values))                             # Создаем список кортежей из парных элементов, где первый элемент - бренд авто, второй - количество фотографий этого бренда
brand_numbers.sort(key = lambda x: x[1], reverse = True)                        # Делаем сортировку списка по количеству фото (второму элементу кортежа)

# Выводим информацию по количеству фотографий для каждого бренда
num_ = 1
for i, j in (brand_numbers):
  print(f'{num_}. Для бренда {i} можно скачать {j} изображений')
  num_ += 1

1. Для бренда Mercedesbenz можно скачать 3077 изображений
2. Для бренда Kia можно скачать 2955 изображений
3. Для бренда Audi можно скачать 2678 изображений
4. Для бренда Hyundai можно скачать 2630 изображений
5. Для бренда Volkswagen можно скачать 2335 изображений
6. Для бренда Toyota можно скачать 2031 изображений
7. Для бренда Nissan можно скачать 1972 изображений
8. Для бренда Bmw можно скачать 1900 изображений
9. Для бренда Renault можно скачать 1795 изображений
10. Для бренда Lexus можно скачать 1778 изображений
11. Для бренда Ford можно скачать 1576 изображений
12. Для бренда Landrover можно скачать 1407 изображений
13. Для бренда Mitsubishi можно скачать 1393 изображений
14. Для бренда Skoda можно скачать 1379 изображений
15. Для бренда Volvo можно скачать 1362 изображений
16. Для бренда Peugeot можно скачать 1286 изображений
17. Для бренда Porsche можно скачать 1245 изображений
18. Для бренда Chevrolet можно скачать 1116 изображений
19. Для бренда Honda можно скачать 1077 изоб

In [ ]:
# Выбор брендов для скачивания изображений
selected_brands = [brand_name for brand_name, brand_num in brand_numbers[79:82]]
selected_brands

['Marussia', 'Maserati', 'Mastretta']

In [ ]:
['country',	'Сколько_вам_лет',	'В_какой_сфере_сейчас_работаете',	'Ваш_средний_доход_в_месяц',	'Рассматриваете_ли_в_перспективе_платное_обучение_профессии_Разработчик_Искусственного_Интеллекта',	'Сколько_времени_готовы_выделить_на_обучение_в_неделю']:

In [ ]:
country, age, job, earnings, training, time

In [ ]:
data_path = './cars_images/'          # Указание пути для сохранения данных
os.makedirs(data_path, exist_ok=True) # Создание папки для данных, если её не существует

In [ ]:
# Сначала спарсим все ссылки на фотографии
start_time = time.time()                                                        # Засекаем время выполнения кода в ячейке
count = 1                                                                       # Счетчик страниц сайта, начинаем извлекать данные с первой страницы
link_dict = {}                                                                  # Словарь для хранения ссылок на фото для брендов {бренд: ссылки}

for brand_name in selected_brands:                                              # Итерация по рассматриваемым брендам
  print(f'{count}. Получение ссылок для бренда: {brand_name}')                  # Информация о процессе
  

  current_link = link_root + brand_dict[brand_name]                             #  Получаем текущую ссылку (пример: http://www.motorpage.ru/Hummer/photos.html)
  
  response = requests.get(current_link)                                         # Делаем запрос на сайт и получаем ответ
  html_soup = BeautifulSoup(response.text)                                      # Получаем html для обработки информации

  model_cards = html_soup.findAll('a', {'class': "col-xs-12 col-sm-4 col-md-3"}) # По тегу и классу получаем блоки с информацией по каждой модели выбранной марки
  model_links = [model_card['href'] for model_card in model_cards]              # Получение ссылок на изображения по каждой марке каждого бренда
  brand_links = []                                                              # Хранилище для ссылок на изображения машин данного бренда

  for model_link in tqdm(model_links):                                          # итерация по всем ссылкам на изображения с моделями
      direct_link = link_root + model_link                                      # Получаем ссылку, пример: http://www.motorpage.ru/Hummer/H2/last/photos/
      response = requests.get(direct_link)                                      # Делаем запрос на сайт
      html_soup = BeautifulSoup(response.text)                                  # Получаем объект для обработки информации
      picture_frames = html_soup.findAll('img', {'title': True})                # Собираем все блоки с тегом img, {'title': True} поможет найти только блоки с изображениями, где у которых есть заголовок
      picture_links = [frame['src'] for frame in picture_frames]                # Сбор ссылок на исходные изображенения 
      brand_links += picture_links                                              # Добавляем в список brand_links все ссылки на изображения по текущей модели авто   
  link_dict[brand_name] = brand_links                                           # Добавляем в словарь link_dict ключ бренда авто и список ссылок на изображения в качестве значения


  # Генерируем задержку до следующего запроса от 1 до 4 секунд
  value = random.random()                                                       # Генерируем случайное число от 0 до 1
  time_sleep = 1 + value*3                                                      # Добавляем случайное число к единице
  print(f'Задержка до следующего запроса: {round(time_sleep, 1)} сек')
  time.sleep(time_sleep)                                                        # Откладываем исполнение кода на time_sleep секунд
  count +=1
  print('--------------------------------')

end_time = time.time() - start_time                                             # Считаем сколько времени ушло на парсинг
print()
print(f'Время извлечения данных: {round(end_time/60, 1)} минут')  

1. Получение ссылок для бренда: Marussia


100%|██████████| 2/2 [00:00<00:00,  2.42it/s]


Задержка до следующего запроса: 3.7 сек
--------------------------------
2. Получение ссылок для бренда: Maserati


100%|██████████| 9/9 [00:03<00:00,  2.47it/s]


Задержка до следующего запроса: 1.1 сек
--------------------------------
3. Получение ссылок для бренда: Mastretta


100%|██████████| 1/1 [00:00<00:00,  1.55it/s]


Задержка до следующего запроса: 3.5 сек
--------------------------------

Время извлечения данных: 0.2 минут


In [ ]:
link_dict

{'Hummer': ['http://cdn.motorpage.ru/Photos/800/4f8e6bf8eca57.jpg',
  'http://cdn.motorpage.ru/Photos/800/4f8e6bf8f08d7.jpg',
  'http://cdn.motorpage.ru/Photos/800/4f8e6bf8eab1c.jpg',
  'http://cdn.motorpage.ru/Photos/800/13C.jpg',
  'http://cdn.motorpage.ru/Photos/800/4f8e6bf8cb6f0.jpg',
  'http://cdn.motorpage.ru/Photos/800/4f8e6bf8b7e9c.jpg',
  'http://cdn.motorpage.ru/Photos/800/4f8e6bf8bebf9.jpg',
  'http://cdn.motorpage.ru/Photos/800/4f8e6bf8d14b1.jpg',
  'http://cdn.motorpage.ru/Photos/800/4f8e6bf8da157.jpg',
  'http://cdn.motorpage.ru/Photos/800/Hummer_I_01_.jpg',
  'http://cdn.motorpage.ru/Photos/800/Hummer_I_01_1.jpg',
  'http://cdn.motorpage.ru/Photos/800/Hummer_I_02_.jpg',
  'http://cdn.motorpage.ru/Photos/800/4f8e6c3d796bd.jpg',
  'http://cdn.motorpage.ru/Photos/800/4f8e6c3d7199e.jpg',
  'http://cdn.motorpage.ru/Photos/800/4f8e6c3d5f0ac.jpg',
  'http://cdn.motorpage.ru/Photos/800/4f8e6c3d5a29b.jpg',
  'http://cdn.motorpage.ru/Photos/800/4f8e6c3d2e350.jpg',
  'http://cdn.mo

In [ ]:
# Скачиваем изображения

start_time = time.time()                                                        # Засекаем время выполнения кода в ячейке
count = 1                                                                       # Счетчик страниц сайта, начинаем извлекать данные с первой страницы

for brand_name, link_list in link_dict.items():                                 # В цикле берем бренд и список ссылок на его изображение
  print(f'{count}. Скачивание изображений для бренда: {brand_name}')            # Выводим инфо процесса
  os.makedirs(data_path + brand_name, exist_ok=True)                            # Создание папки бренда, если её не существует
  dir_path = data_path + brand_name                                             # Указываем текущую директорию, куда сохраняем изображения

  for idx, link in enumerate(tqdm(link_list)):                                  # Проходим по каждой ссылке
    urllib.request.urlretrieve(link, f"{dir_path}/{'0'*(4-len(str(idx)))}{idx}_{brand_name}.jpg") # Сохраняем изображение

  count +=1
  print()
  print('--------------------------------')

end_time = time.time() - start_time                                             # Считаем сколько времени ушло на парсинг
print()
print(f'Время скачивания: {round(end_time/60, 1)} минут')      

  0%|          | 0/11 [00:00<?, ?it/s]

1. Скачивание изображений для бренда: Marussia


  0%|          | 0/150 [00:00<?, ?it/s]


--------------------------------
2. Скачивание изображений для бренда: Maserati


  0%|          | 0/6 [00:00<?, ?it/s]


--------------------------------
3. Скачивание изображений для бренда: Mastretta


100%|██████████| 6/6 [00:03<00:00,  1.59it/s]


--------------------------------

Время скачивания: 1.7 минут


# Получение данных с kinopoisk.ru через API

**Что такое API**

API (Application programming interface) — это контракт, который предоставляет программа. «Ко мне можно обращаться так и так, я обязуюсь делать то и это».

Подробнее: https://habr.com/ru/post/464261/

https://kinopoisk.cloud/

In [ ]:
# Получаем и изучаем json файл

api_token='Ваш Токен'   # Токен, доступный после регистрации

url = f'https://api.kinopoisk.cloud/movies/{1143242}/token/Ваш Токен'    # Url для запроса данных с сайта
request = get(url)              # Отправляем запрос, получаем ответ
data = request.json()           # Преобразуем ответ в json
data

{'actors': ['Мэттью МакКонахи',
  'Чарли Ханнэм',
  'Генри Голдинг',
  'Хью Грант',
  'Мишель Докери',
  'Джереми Стронг',
  'Эдди Марсан',
  'Джейсон Вонг',
  'Колин Фаррелл',
  'Лайн Рени',
  'Василий Дахненко',
  'Сергей Смирнов',
  'Филипп Бледный',
  'Алексей Иващенко',
  'Татьяна Шитова'],
 'age': 'зрителям, достигшим 18 лет',
 'budget': None,
 'collapse': {'duration': ['113 мин. / 01:53'],
  'quality': ['FHD (1080p)'],
  'url': ['/embed/movie/14178'],
  'voice': ['Рус. Дублированный', 'Укр. Дубльований', 'Eng.Original']},
 'composers': ['Кристофер Бенстед'],
 'countries': ['США'],
 'description': 'Талантливый выпускник Оксфорда, применив свой уникальный ум\xa0и невиданную дерзость, придумал нелегальную схему обогащения с\xa0использованием поместья обедневшей английской аристократии. Однако когда он\xa0решает продать свой бизнес влиятельному клану миллиардеров из\xa0США, на\xa0его пути встают не\xa0менее обаятельные, но\xa0жесткие джентльмены. Намечается обмен любезностями, котор

In [ ]:
# Пример отсутствия фильма по id

url = f'https://api.kinopoisk.cloud/movies/{1143244}/token/c9972b77e92bbc443c28279e1a29ec58'
request = get(url) # Отправляем запрос, получаем ответ
data = request.json() 
data

{'error': {'code': 4, 'message': 'The movie is not in the database.'}}

In [ ]:
start_time = time.time() # Засекаем время выполнения кода в ячейке

films_data = [] # Список для хранения данных по всем фильмам

# Названия ключей json файла, к кторым будем обращаться для получения нужных данных
params = ['title', 'id_kinopoisk', 'rating_imdb', 'rating_kinopoisk', 'year', 'actors', 'budget', 'composers', 'directors', 'fees_world', 'genres', 'description']
skip_films = 0 # Счетчик пропущенных id
count = 1 # Счетчик итераций

for movie_number in range(1143242, 1143242+90):  # Перебираем id фильмов
    try:
        url = f'https://api.kinopoisk.cloud/movies/{movie_number}/token/c9972b77e92bbc443c28279e1a29ec58' # Передаем id фильма в url
        request = get(url)  # Отправляем запрос, получаем ответ
        data = request.json()  # Получаем json файл

        print(f"{count}. Получение информации для фильма '{data['title']}'") # Информация о процессе
        temp_list = []                                    # Создаем временный список, куда вносим данные по текущему фильму
        for param in params:                              # Проходимся по нужным нам ключам в json файле
           temp_list.append(data[param])                  # Добавляем данные во временный список
        temp_list.append(data['collapse']['duration'])    # Отдельно добавляем информацию по длительности фильма
        films_data.append(temp_list)                      # В список films_data добалвяем как элемент временный список temp_list
        count += 1

    except KeyError:                                      # В случае, если фильма нет, выводим об этом сообщение
        print(f'{count} фильм с id {movie_number} отсутствует в базе')
        count += 1
        skip_films += 1 

end_time = time.time() - start_time                                         # Считаем сколько времени ушло на парсинг
print()
print(f'Время извлечения данных: {round(end_time/60, 1)} минут')
print(f'Пропущено {skip_films} id')   

1. Получение информации для фильма 'Джентльмены'
2. Получение информации для фильма 'Schoolies Week Murders, The'
3 фильм с id 1143244 отсутствует в базе
4. Получение информации для фильма 'Viewing, The'
5. Получение информации для фильма 'Days in London'
6. Получение информации для фильма 'Sam Tsui & Alyson Stoner: Despacito'
7 фильм с id 1143248 отсутствует в базе
8. Получение информации для фильма 'Dalulana Gini'
9 фильм с id 1143250 отсутствует в базе
10. Получение информации для фильма 'Congratulations! You Have a New Match!'
11. Получение информации для фильма 'Kiss of the Blarney Stone'
12 фильм с id 1143253 отсутствует в базе
13 фильм с id 1143254 отсутствует в базе
14 фильм с id 1143255 отсутствует в базе
15 фильм с id 1143256 отсутствует в базе
16. Получение информации для фильма '21 Files'
17 фильм с id 1143258 отсутствует в базе
18 фильм с id 1143259 отсутствует в базе
19. Получение информации для фильма 'Cross-Stitch'
20 фильм с id 1143261 отсутствует в базе
21. Получение 

In [ ]:
df = pd.DataFrame(films_data, columns=['title', 'id', 'Рейтинг IMDB', 'Рейтинг Кинопоиск', 'Год', 'Актёры', 'Бюджет', 'Композиторы', 'Режиссёр', 'Сборы', 'Жанры', 'Описание', 'Длительность']) # создаем датафрейм

In [ ]:
df

,title,id,Рейтинг IMDB,Рейтинг Кинопоиск,Год,Актёры,Бюджет,Композиторы,Режиссёр,Сборы,Жанры,Описание,Длительность
0,Джентльмены,1143242,7.9,8.5,2019.0,"[Мэттью МакКонахи, Чарли Ханнэм, Генри Голдинг...",None,[Кристофер Бенстед],[Гай Ричи],$87528459,"[Боевик, Комедия, Криминал]","Талантливый выпускник Оксфорда, применив свой ...",[113 мин. / 01:53]
1,"Schoolies Week Murders, The",1143243,NaN,NaN,2017.0,"[Jeni Bezuidenhout, Frances McCarthy, Nathan S...",None,None,None,None,"[Короткометражка, Ужасы]",None,None
2,"Viewing, The",1143245,NaN,NaN,2018.0,"[Sharon Spink, Laurence J. Campling, Рэй Микаэ...",None,None,[Sebastian Roberts],None,"[Короткометражка, Ужасы]",None,None
3,Days in London,1143246,NaN,NaN,2018.0,"[Elisa Balau, Elisa Bernardini, Viola Bernardi...",None,[Elisa Bernardini],[Elisa Bernardini],None,"[Короткометражка, Музыка]",None,None
4,Sam Tsui & Alyson Stoner: Despacito,1143247,NaN,NaN,2017.0,"[Элисон Стоунер, Сэм Цуй]",None,None,None,None,"[Короткометражка, Музыка]",None,None
5,Dalulana Gini,1143249,NaN,NaN,1995.0,"[Damith Fonseka, Dilhani Ekanayake, Cleatus Me...",None,None,None,None,[Боевик],None,None
6,Congratulations! You Have a New Match!,1143251,5.3,NaN,2018.0,"[Emilia Jansson, Joni Kainulainen, Tuomas Kain...",None,[Юсси Хатала],[Rauli Ylitalo],None,"[Короткометражка, Драма]",None,None
7,Kiss of the Blarney Stone,1143252,NaN,NaN,2018.0,"[Дэйв Р. Уоткинс, Кендес Мэбри, Grant Garlingh...",None,[Markleford Friedman],[Дэйв Р. Уоткинс],None,"[Короткометражка, Комедия]",None,None
8,21 Files,1143257,NaN,NaN,2019.0,None,None,None,None,None,None,None,None
9,Cross-Stitch,1143260,NaN,NaN,2017.0,None,None,[Carter Knopik],[Maya Peirce],None,"[Короткометражка, Драма]",None,None
